In [2]:
import pandas as pd
from __future__ import print_function
print('All modules have been imported')

All modules have been imported


In [ ]:
#Grocery Dataset
#format: each row is an example
#we will choose the last row as the label (Reorder or not)
#the first four columns will be the features (dept, dow, hour, weeks_prior, label) 

training_data = [
    ['processed food','weekday','before noon','2', 1 ],
    ['fresh food','weekday','after noon','2+', 1 ],
    ['fresh food','weekday','after noon','1', 1 ],
    ['processed food','weekday','before noon','2+', 1 ],
    ['fresh food','weekend','after noon','2+', 1 ],
    ['fresh food','weekday','before noon','2+', 0 ],
    ['processed food','weekday','before noon','2+', 0 ],
    ['processed food','weekday','after noon','2+', 0 ],

]

#removed miscellaneous as it only leads to a leaf node, spoiling the model

In [61]:
#Labeling the columns
header = ['Department', 'Day of the week', 'Hour of the day', 'Weeks prior', 'Label']

In [62]:
#testing for unique values present with a function
def unique(rows,col):
    return set([row[col] for row in rows])

In [63]:
#Testing the unique function:
unique(training_data, 0)
unique(training_data, 1)
unique(training_data, 2)
#out put is returned in the form of a set since we do not want a repeat value


{'after noon', 'before noon'}

In [64]:
#counting the number of reorders
def count(rows):
    counts = {} #dictionary always makes counting easy of the labels
    for row in rows:

        label = row[-1] #our label(reorder or not) is the last column
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts


In [65]:
#Testing the count function:
count(training_data) #successful

{1: 5, 0: 3}

In [66]:
class Question:
    #A decision tree will ask questions to classify a dataset
    #Our class will record a column number (e.g 0 for department) and three column values 
    #(e.g fresh, processed or misc.)
    #A match method will be used to compare the feature value in an example with a feature value
    #stored in the question

    def __init__(self, column, value):

        self.column = column
        self.value = value

    def match(self, example):
        #Will be used to compare a feature in an example to a feature in the question
        val = example[self.column]
        return val == self.value


    def __repr__(self):
        #prints the question in a readable format
        condition = '=='
        return "Is %s %s %s ?" % (header[self.column],condition, self.value)


In [67]:
Question(0,'processed food') #testing out if the question works!

Is Department == processed food ?

In [68]:
q = Question(0, 'processed food') #another example 
q

Is Department == processed food ?

In [69]:
#Now the big moment, testing with an example to match a feature
example = training_data[0]

q.match(example) #Success, it returns True since the first item which is a list in itself in the training 
#data has 'processed food present in it

True

In [70]:
#Partition
#function to partition the dataset in true(example matches the question) and false(example not match the question)
def partition(rows, question):
    true, false = [], []
    for row in rows:
        if question.match(row):
            true.append(row)
        else:
            false.append(row)
    return true, false


In [71]:
#we will test if the function partition works!
#we will partition based on weather the row has processed food
true, false = partition(training_data, Question(0, 'processed food'))
#check for true rows: should have processed food
true

[['processed food', 'weekday', 'before noon', '2', 1],
 ['processed food', 'weekday', 'before noon', '2+', 1],
 ['processed food', 'weekday', 'before noon', '2+', 0],
 ['processed food', 'weekday', 'after noon', '2+', 0]]

In [72]:
#The following rows should not have anything related to processed food
false

[['fresh food', 'weekday', 'after noon', '2+', 1],
 ['fresh food', 'weekday', 'after noon', '1', 1],
 ['fresh food', 'weekend', 'after noon', '2+', 1],
 ['fresh food', 'weekday', 'before noon', '2+', 0]]

In [73]:
#gini calculations
#calculating the gini impurity for a list of rows
def gini(rows):

    counting = count(rows)
    impurity = 1
    for i in counting:
        prob = counting[i] / float(len(rows))
        impurity -= prob**2
    return impurity

In [74]:
#testing out the gini function
pure = [['fresh food'],['fresh food']]
#should return a gini impurity of 0
gini(pure)

0.0

In [75]:
#impure sample testing
impure = [['fresh food'], ['processed food'], ['miscellaneous']]
gini(impure)
#should return a value !=0

0.6666666666666665

In [76]:
def info_gain(left, right, present_uncertainity):
    #Information gain = uncertainity of root node minus the weighted impurity of two child nodes

    p = float(len(left)) / (len(left) + len(right))
    return present_uncertainity - p * gini(left) - (1-p) * gini(right)


In [77]:
#testing the uncertainity of the training set
present_uncertainity = gini(training_data)
present_uncertainity

0.46875

In [78]:
#how much information we will gain by partitioning on processed food?
true, false = partition(training_data,Question(0, 'processed food'))
info_gain(true, false, present_uncertainity)

0.03125

In [79]:
#what if partitioned on fresh food?
true, false = partition(training_data,Question(0, 'fresh food'))
info_gain(true, false, present_uncertainity)

0.03125

In [80]:
true, false = partition(training_data,Question(0, 'miscellaneous'))
info_gain(true, false, present_uncertainity)

0.0

In [81]:
#from the above calculations, we see we gained more information from the miscellaneous
#we can look for the rows that give us more jumbled up data

true, false = partition(training_data,Question(0, 'miscellaneous'))
true
#true rows only have no reorder! # clean and easy to separate data

[]

In [82]:
false

[['processed food', 'weekday', 'before noon', '2', 1],
 ['fresh food', 'weekday', 'after noon', '2+', 1],
 ['fresh food', 'weekday', 'after noon', '1', 1],
 ['processed food', 'weekday', 'before noon', '2+', 1],
 ['fresh food', 'weekend', 'after noon', '2+', 1],
 ['fresh food', 'weekday', 'before noon', '2+', 0],
 ['processed food', 'weekday', 'before noon', '2+', 0],
 ['processed food', 'weekday', 'after noon', '2+', 0]]

In [83]:
true, false = partition(training_data,Question(0, 'fresh food'))
true

[['fresh food', 'weekday', 'after noon', '2+', 1],
 ['fresh food', 'weekday', 'after noon', '1', 1],
 ['fresh food', 'weekend', 'after noon', '2+', 1],
 ['fresh food', 'weekday', 'before noon', '2+', 0]]

In [84]:
true, false = partition(training_data,Question(0, 'processed food'))
true

[['processed food', 'weekday', 'before noon', '2', 1],
 ['processed food', 'weekday', 'before noon', '2+', 1],
 ['processed food', 'weekday', 'before noon', '2+', 0],
 ['processed food', 'weekday', 'after noon', '2+', 0]]

In [85]:
#Time for the real game!
def best_split(rows):
    #we will find the best question to ask by going over every feature and calculate the information gain
    best_gain = 0 # initialized and tracking the best information gain
    best_question = None #initialized and keeping track of the feature/value that produced it
    present_uncertainity = gini(rows)
    n_features = len(rows[0]) - 1 #number of columns

    for col in range(n_features): #iterate over every feature

        values = set([row[col] for row in rows]) #for the different values of the column
        for val in values: #every value considered
            question = Question(col, val)

            #split the dataset
            true, false = partition(rows, question)
            #we shall skip the split if the dataset is not not divided
            if len(true) == 0 or len(false) == 0:
                continue
            #calculate the info gain from the split
            gain = info_gain(true, false, present_uncertainity)

            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question


In [86]:
#Real test! What is the best question to ask for our dataset?
best_gain, best_question = best_split(training_data)
best_question

Is Weeks prior == 2+ ?

In [87]:
class Leaf:
    #leaf node to classify the data, number of times a value will occur in each row such as processed food

    def __init__(self, rows):
        self.predictions = count(rows)

In [88]:
class Decision_Node:
    #the node that will ask the questions!
    #will lead to the division into child nodes

    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self. true_branch = true_branch
        self.false_branch = false_branch

In [89]:
def build_tree(rows):
    #we shall try to partition the dataset on unique attributes, calculate the info gain and return the 
    #question which has the highest gain

    gain, question = best_split(rows)

    #base case: when there is no info gain, no questions to ask
    # a leaf node is returned
    if gain == 0:
        return Leaf(rows)

    #From here, useful information feature found that can be used to partition
    true, false = partition(rows, question)

    #RECURSION: building the true branch
    true_branch = build_tree(true)

    #RECURSION: building the false branch
    false_branch = build_tree(false)

    #Lastly we shall return the Decision node, that will have the best question to ask at the point
    #depending on the answer

    return Decision_Node(question, true_branch, false_branch)


In [90]:
def print_tree(node, spacing = ''):
    #formating the printing of the decision tree

    #Base case:
    if isinstance(node, Leaf):
        print(spacing + 'Predict', node.predictions)
        return

    #printing of question at the node
    print(spacing + str(node.question))

    #recursion on the true branch
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + '  ')

    #recursion on the false branch
    print(spacing + '--> False:')
    print_tree(node.true_branch, spacing + '  ')


In [91]:
my_tree = build_tree(training_data)

In [92]:
print_tree(my_tree)

Is Weeks prior == 2+ ?
--> True:
  Is Day of the week == weekend ?
  --> True:
    Predict {1: 1}
  --> False:
    Predict {1: 1}
--> False:
  Is Day of the week == weekend ?
  --> True:
    Predict {1: 1}
  --> False:
    Predict {1: 1}


In [93]:
def classify(row, node):

    #Base case: when we reach a leaf node
    if isinstance(node, Leaf):
        return node.predictions

    #whether to go to the true branch or false branch?
    #will compare feature stored in the node to the example

    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
        

In [ ]:
#testing!
classify(training_data[0], my_tree)

In [95]:
#making probabilities:
def print_leaf(count):

    total = sum(count.values()) * 1
    probs = {}
    for lbl in count.keys():
        probs[lbl] = str(int(count[lbl] / total * 100)) + '%'
    return probs

In [97]:
#testing
print_leaf(classify(training_data[0], my_tree))

{1: '100%'}

In [102]:
#Evaluation:
testing_data = [
    ['fresh food','weekend','after noon','2+', 0 ],
    ['fresh food','weekend','after noon','2+',1],
    ['fresh food','weekday','before noon','2',1],
    ['fresh food','weekday','before noon','2',1],
    ['fresh food','weekday','before noon','2',1],
    ['processed food','weekday','before noon','2',1],
    ['processed food','weekday','before noon','2',1],
    ['processed food','weekday','before noon','2',1],
    ['processed food','weekday','before noon','2',1],
    ['processed food','weekday','before noon','2',1],
    ['processed food','weekday','before noon','2',1],
    ['processed food','weekday','before noon','2',1],
    ['processed food','weekday','before noon','2',1],
    ['processed food','weekday','before noon','2',1],
    ['processed food','weekend','before noon','2',0],
    ['processed food','weekend','before noon','2',0],
    ['processed food','weekend','after noon','2',1],
    ['processed food','weekday','before noon','2+',0],
    ['processed food','weekday','after noon','2+',0],
    ['processed food','weekday','after noon','2+',0],
    ['processed food','weekend','before noon','2+',0],
    ['processed food','weekend','before noon','2+',0],
    ['processed food','weekday','before noon','2',1],
    ['processed food','weekend','before noon','2+',0],
    ['processed food','weekend','before noon','2',0],
    ['processed food','weekend','after noon','2',0],
    ['processed food','weekend','after noon','2',1],
    ['processed food','weekday','after noon','2+',0],
    ['processed food','weekday','after noon','2+',0],
    ['processed food','weekday','after noon','2+',0],
    ['fresh food','weekend','after noon','2+',0],
    ['fresh food','weekend','after noon','2+',0],
    ['fresh food','weekend','after noon','2+',0],
    ['fresh food','weekend','after noon','2+',0],
    ['fresh food','weekend','after noon','2+',0],
    ['fresh food','weekend','after noon','2+',1],
    ['fresh food','weekend','after noon','2+',1],
    ['fresh food','weekend','after noon','2+',1],
    ['fresh food','weekday','before noon','2',1],
    ['fresh food','weekday','before noon','2',1],
    ['fresh food','weekday','before noon','2',1],
    ['fresh food','weekday','before noon','2',1],
    ['fresh food','weekday','before noon','2',1],
    ['fresh food','weekday','before noon','2',1],
    ['fresh food','weekday','before noon','2',1],
    ['fresh food','weekend','before noon','2+',0],
    ['fresh food','weekday','before noon','1',1],
    ['fresh food','weekday','before noon','1',1],
    ['fresh food','weekday','before noon','1',1],
    ['processed food','weekend','after noon','2+',0],
    ['processed food','weekday','before noon','2',1],
]

In [ ]:
for row in testing_data:
    print('Actual: %s. Predicted: %s' %
    (row[-1], print_leaf(classify(row, my_tree))))

In [104]:
print((35/50)*100)

70.0
